In [ ]:
# Install packages
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 20.9 MB/s eta 0:00:00


In [ ]:
# Import plotting packages
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

# Import other packages
from nltk.tokenize import RegexpTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from gensim.models import CoherenceModel
import re
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.8/dist-packages/gensim/corpora/dictionary.py:11: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping, defaultdict
/usr/local/lib/python3.8/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [ ]:
# read in data to google colab (only use this code block if running on Google Colab)
from google.colab import files
uploades = files.upload()

Saving state_ofthe_union_texts.csv to state_ofthe_union_texts.csv


In [ ]:
# Load in state of the union address data
import pandas as pd
# Read in data
url = "/content/state_ofthe_union_texts.csv"
df = pd.read_csv(url)

In [ ]:
# Create a WordCloud to see possible stop words

# See reference [1]

# remove punctuation
df['Text_processed'] = \
df['Text'].map(lambda x: re.sub('[,\.!?]', '', x))

# convert titles to lowercase
df['Text_processed'] = \
df['Text_processed'].map(lambda x: x.lower())

# join text together
long_string = ','.join(list(df['Text_processed'].values))

# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=1000, contour_width=3, contour_color='steelblue')

# Generate a word cloud
wordcloud.generate(long_string)

# Visualize the word cloud
wordcloud.to_image()

In [ ]:
# See reference [2]
# Tokenize
tokenizer = RegexpTokenizer(r'\w+') # Separate the sentence into words without punctuation

# Use english stop wrods from stopwords package
stop_words = stopwords.words('english')
# Append stop words that are specific to the State of the Union Addresses
stop_words.extend(['shall', 'applause', 'nation', 'year', 'must', 'world', 
                   'people', 'america', 'american', 'us', 'congress','u', 'y'])

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

# Topic Modeling for all SOTU addresses (SOTU addresses as a whole)

# Compile speech documents into a list
doc_set = df['Text'].values.tolist()

# Tokenize speeches
texts = []

# Loop through speech document list
for i in doc_set:
    
    # Clean and tokenize each speech document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # Remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in stop_words]
    
    # Stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # Add tokens to list
    texts.append(stemmed_tokens)


# Turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

# Filter extremes
# No tokens appearing in less than 5 speeches, no tokens in above 50% of speeches
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

# Convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# Generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                           num_topics=3, 
                                           id2word = dictionary, 
                                           passes=100)

# Print topics from the model
for idx, topic in ldamodel.print_topics():
  print("Topic: {} \nWords: {}".format(idx, topic ))
  print("\n")

In [ ]:
# Coherence of entire corpus, not each topic for each speech

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=texts, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.5303897251003954


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.130326 -0.083450       1        1  56.848988
0      0.178359 -0.030334       2        1  26.564386
2     -0.048033  0.113784       3        1  16.586627, topic_info=          Term         Freq        Total Category  logprob  loglift
5748   program  1154.000000  1154.000000  Default  30.0000  30.0000
4564       job   729.000000   729.000000  Default  29.0000  29.0000
1605  american   797.000000   797.000000  Default  28.0000  28.0000
2595      nthe   398.000000   398.000000  Default  27.0000  27.0000
5285       nof   383.000000   383.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
2397       per   168.811883   659.690382   Topic3  -5.6794   0.4336
178   function   111.677624   238.174801   Topic3  -6.0926   1.0392
1782     canal   127.731743   365.753521   Topic3  -5.9582   0.7445
1481   project   109.176960   286.737660   Topic3  -6.1152   0.8310
5748   program   107.350382  1154.364355   Topic3  -6.1321  -0.5786

[219 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
2410      1  0.992272      01
5055      1  0.991874    1890
5429      1  0.984055    1893
6074      2  0.994890    1946
6075      2  0.994531    1947
...     ...       ...     ...
1044      2  0.822601   women
1044      3  0.109225   women
5329      1  0.010234  worker
5329      2  0.874981  worker
5329      3  0.115129  worker

[366 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

##References
[1] S. Kapadia. "Topic Modeling in Python: Latent Dirichlet Allocation (LDA)." Medium. https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0 (retrieved Feb. 19, 2023).

[2] J. Barber. "Latent Dirichlet Allocation (LDA) with Python." AmazonAWS. https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html (retrieved Feb. 19, 2023).